参考:  
https://blog.floydhub.com/a-beginners-guide-on-recurrent-neural-networks-with-pytorch/  
https://github.com/gabrielloye/RNN-walkthrough/blob/master/main.ipynb

In [1]:
import torch
from torch import nn
import numpy as np

# テキストの準備・前処理

まずはテキスト、文字と数字の対応を定義します。

In [31]:
text=["hey how are you","good i am fine","have a nice day"] # テキスト
print(text)

chars=set("".join(text)) # ユニークな文字を取得
print(chars)

int2char=dict(enumerate(chars)) # 数字→文字の辞書を形成
print(int2char)

char2int={char:ind for ind,char in int2char.items()} # 文字→数字の辞書を形成
print(char2int)

['hey how are you', 'good i am fine', 'have a nice day']
{'o', 'c', 'a', 'd', 'v', 'h', 'f', ' ', 'y', 'n', 'r', 'u', 'g', 'm', 'e', 'w', 'i'}
{0: 'o', 1: 'c', 2: 'a', 3: 'd', 4: 'v', 5: 'h', 6: 'f', 7: ' ', 8: 'y', 9: 'n', 10: 'r', 11: 'u', 12: 'g', 13: 'm', 14: 'e', 15: 'w', 16: 'i'}
{'o': 0, 'c': 1, 'a': 2, 'd': 3, 'v': 4, 'h': 5, 'f': 6, ' ': 7, 'y': 8, 'n': 9, 'r': 10, 'u': 11, 'g': 12, 'm': 13, 'e': 14, 'w': 15, 'i': 16}


のちにテキストの長さを最も長いものに揃えるために、最大の長さを取得します。

In [24]:
maxlen=len(max(text,key=len)) # テキストの最大の長さ
print("The longest string has {} characters".format(maxlen))

The longest string has 15 characters


In [25]:
for i in range(len(text)):# 各テキストについて長さとテキストを表示
    print(len(text[i]),text[i])

15 hey how are you
14 good i am fine
15 have a nice day


テキストの文字数を揃えるために、短いテキストの末尾に" "（半角スペース）を追加します。

In [26]:
for i in range(len(text)): # 各テキストについて　
    while len(text[i])<maxlen:　# 一番長いものに合うように
        text[i]+=" " # 末尾にスペースを追加

for i in range(len(text)): # 各テキストについて長さとテキストを表示
    print(len(text[i]),text[i])

15 hey how are you
15 good i am fine 
15 have a nice day


テキストを入力用と出力用の2つ用意します。

In [29]:
input_seq=[] # 入力とするシーケンス
target_seq=[] # 出力とするシーケンス

for i in range(len(text)): # 各テキストについて
    input_seq.append(text[i][:-1]) # 入力
    target_seq.append(text[i][1:]) # 出力
    print("Input Sequence: {}\n\tTarget Sequence: {}".format(input_seq[i],target_seq[i]))

Input Sequence: hey how are yo
	Target Sequence: ey how are you
Input Sequence: good i am fine
	Target Sequence: ood i am fine 
Input Sequence: have a nice da
	Target Sequence: ave a nice day


各文字を数字に変換してゆきます。

In [30]:
for i in range(len(text)):
    input_seq[i]=[char2int[character] for character in input_seq[i]] # 数字に変換したテキスト
    target_seq[i]=[char2int[character] for character in target_seq[i]] # 数字に変換したテキスト
    print("Input Sequence: {}\n\tTarget Sequence: {}".format(input_seq[i],target_seq[i]))

Input Sequence: [5, 14, 8, 7, 5, 0, 15, 7, 2, 10, 14, 7, 8, 0]
	Target Sequence: [14, 8, 7, 5, 0, 15, 7, 2, 10, 14, 7, 8, 0, 11]
Input Sequence: [12, 0, 0, 3, 7, 16, 7, 2, 13, 7, 6, 16, 9, 14]
	Target Sequence: [0, 0, 3, 7, 16, 7, 2, 13, 7, 6, 16, 9, 14, 7]
Input Sequence: [5, 2, 4, 14, 7, 2, 7, 9, 16, 1, 14, 7, 3, 2]
	Target Sequence: [2, 4, 14, 7, 2, 7, 9, 16, 1, 14, 7, 3, 2, 8]


各シーケンスをone-hotに変換する

In [32]:
def one_hot_encode(sequence,dict_size,seq_len,batch_size):
    features=np.zeros((batch_size,seq_len,dict_size),dtype=np.float32) # 特徴量行列の初期化
    print("features.shape = ",features.shape)
    
    for i in range(batch_size): #  各バッチについて
        for u in range(seq_len): # 各シーケンスについて
            features[i,u,sequence[i][u]]=1 # 対応する文字だけ1にする
        
        print(features[i,:,:])
            
    return features

それではone-hotに変換します。

In [33]:
dict_size=len(char2int) # 辞書のサイズ
seq_len=maxlen-1 # シーケンスの長さ
batch_size=len(text) # バッチサイズ

input_seq=one_hot_encode(input_seq,dict_size,seq_len,batch_size) # シーケンスをone-hotに変換する
print("Input shape: {} --> (Batch Size, Sequence Length, One-Hot Encoding Size)".format(input_seq.shape))

features.shape =  (3, 14, 17)
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

ここからPyTorchを使って計算していきます。

In [34]:
input_seq=torch.from_numpy(input_seq) # 入力をPyTorchの形式に変換
target_seq=torch.Tensor(target_seq) # ターゲットをPyTorchに変換

GPUを使える場合GPUを、そうでない場合はCPUを使うように設定します。

In [36]:
is_cuda=torch.cuda.is_available() # GPUを使えるかどうか

if is_cuda: # GPUを使える場合
    device=torch.device("cuda") # デバイスをGPUに設定
    print("GPU is available")
else:
    device=torch.device("cpu") # デバイスをCPUに設定
    print("GPU is not available, CPU used")

GPU is not available, CPU used


# モデルの定義

モデルを定義します。

In [37]:
class Model(nn.Module):
    def __init__(self,input_size,output_size,hidden_dim,n_layers):
        super(Model,self).__init__()
        
        self.hidden_dim=hidden_dim # 隠れ層の次元
        self.n_layers=n_layers # 層の数
        self.rnn=nn.RNN(input_size,hidden_dim,n_layers,batch_first=True) # RNNレイヤ
        self.fc=nn.Linear(hidden_dim,output_size) # 全結合層
        
    def forward(self,x):
        batch_size=x.size(0) # バッチサイズ
        hidden=self.init_hidden(batch_size) # 隠れ層を初期化する
        out,hidden=self.rnn(x,hidden) # 入力と隠れ層の状態をモデルの入力とし、出力を得る
        out=out.contiguous().view(-1,self.hidden_dim) # 出力の形を変換
        out=self.fc(out) # 全結合層で形を変換
        
        return out,hidden
    
    def init_hidden(self,batch_size):
        hidden=torch.zeros(self.n_layers,batch_size,self.hidden_dim).to(device) # 隠れ層を初期化、デバイスに送る
        
        return hidden

# モデルの学習

それでは学習のための設定をします。

In [38]:
model=Model(input_size=dict_size,output_size=dict_size,hidden_dim=12,n_layers=1) # モデルのインスタンス
model=model.to(device) # モデルをデバイスに送る

n_epochs=100 # エポック数
lr=0.01 # 学習率

criterion=nn.CrossEntropyLoss() # クロスエントロピー誤差
optimizer=torch.optim.Adam(model.parameters(),lr=lr) # 最適化関数

モデルを学習させます。

In [40]:
input_seq=input_seq.to(device) # 入力シーケンス

for epoch in range(1,n_epochs+1): # 各エポックで
    optimizer.zero_grad() # 勾配をゼロにする
    output,hidden=model(input_seq) # モデルの出力を獲得
    output=output.to(device) # 出力をデバイスへ
    target_seq=target_seq.to(device) # ターゲットをデバイスへ
    loss=criterion(output,target_seq.view(-1).long()) # ロスを計算
    loss.backward() # 逆伝搬させる
    optimizer.step() # 重みを更新する
    
    if epoch%10==0: # 10エポックごとに
        print("Epoch: {}/{}..........".format(epoch,n_epochs),end=" ")
        print("Loss: {:.4f}".format(loss.item()))

Epoch: 10/100.......... Loss: 2.4289
Epoch: 20/100.......... Loss: 2.1767
Epoch: 30/100.......... Loss: 1.8516
Epoch: 40/100.......... Loss: 1.4717
Epoch: 50/100.......... Loss: 1.1101
Epoch: 60/100.......... Loss: 0.7869
Epoch: 70/100.......... Loss: 0.5266
Epoch: 80/100.......... Loss: 0.3546
Epoch: 90/100.......... Loss: 0.2520
Epoch: 100/100.......... Loss: 0.1929


# モデルの評価

予測する関数を定義します。

In [41]:
def predict(model,character):
    character=np.array([[char2int[c] for c in character]]) # 各文字を数字に変換
    character=one_hot_encode(character,dict_size,character.shape[1],1) # one-hotに変換
    character=torch.from_numpy(character) # pytorchの肩に変換
    character=character.to(device) # デバイスに送る
    out,hidden=model(character) # モデルの出力
    prob=nn.functional.softmax(out[-1],dim=0).data # ソフトマックスで変換
    char_ind=torch.max(prob,dim=0)[1].item() # 最もスコアの高いクラスを予測クラスとする
    
    return int2char[char_ind],hidden

In [42]:
def sample(model,out_len,start="hey"):
    model.eval() # 評価用モード、batch normalizationとかdropoutとかに関係あるらしい
    start=start.lower() # 小文字に
    chars=[ch for ch in start] # 最初はあらかじめ指定した文字について
    size=out_len-len(chars) # サイズ
    
    for ii in range(size): # サイズ
        char,h=predict(model,chars) # モデルの出力クラスを取得
        chars.append(char) # 文章の末尾に追加
        
    return "".join(chars)

In [43]:
sample(model,15,start="good")

features.shape =  (1, 4, 17)
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
features.shape =  (1, 5, 17)
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
features.shape =  (1, 6, 17)
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
features.shape =  (1, 7, 17)
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 

'good i am fine '